In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemini-generated-2/gemini_dataset.csv
/kaggle/input/epoch-10/weights.pth
/kaggle/input/gemini-data/gemini_generated.csv
/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv
/kaggle/input/new-parameter/weights (1).pth


In [2]:
from transformers import BertModel, BertTokenizer, BertForSequenceClassification

# 加载预训练的模型和分词器
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# 保存模型和分词器
model.save_pretrained('/kaggle/working/Bert_model')
tokenizer.save_pretrained('/kaggle/working/Bert_tokenizer')


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

('/kaggle/working/Bert_tokenizer/tokenizer_config.json',
 '/kaggle/working/Bert_tokenizer/special_tokens_map.json',
 '/kaggle/working/Bert_tokenizer/vocab.txt',
 '/kaggle/working/Bert_tokenizer/added_tokens.json')

In [3]:
import torch
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)

cuda


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [4]:
from sklearn.model_selection import train_test_split
df_train = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv', usecols=['text', 'generated'])
df_gemeni_1 = pd.read_csv('/kaggle/input/gemini-data/gemini_generated.csv', usecols=['text', 'generated'])
df_gemeni_2 = pd.read_csv('/kaggle/input/gemini-generated-2/gemini_dataset.csv', usecols=['text', 'generated'])
texts = df_train['text'].tolist()
texts = texts + df_gemeni_1['text'].tolist() + df_gemeni_2['text'].tolist()
print(len(texts))
labels = df_train['generated'].tolist()
labels = labels + df_gemeni_1['generated'].tolist() + df_gemeni_2['generated'].tolist()
print(len(labels))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


3376
3376


In [5]:
import random

# 生成一个索引列表
indexes = list(range(len(labels)))

# 打乱索引列表
random.shuffle(indexes)

# 使用相同的索引列表来重新排列两个原始列表
shuffled_texts = [texts[i] for i in indexes]
shuffled_labels = [labels[i] for i in indexes]
print(shuffled_labels)

# 拆分数据集
train_texts, val_texts, train_labels, val_labels = train_test_split(shuffled_texts, shuffled_labels, test_size=0.1)

[1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 

In [6]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [7]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(val_encodings, val_labels)

In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

UsageError: api_key not configured (no-tty). call wandb.login(key=[your_api_key])

In [ ]:
def predict(text, model, tokenizer, device):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model(**inputs)
    probs = outputs.logits.softmax(dim=-1)
    return probs[:, 1].item()  # 返回是LLM生成的概率


def process_batch(batch, model, tokenizer, device):
    # 将模型移至正确的设备
    model.to(device)

    # 对批次中的每个文本进行预测
    predictions = []
    for text in batch['text']:
        prediction = predict(text, model, tokenizer, device)
        predictions.append(prediction)

    return predictions



In [ ]:
# 训练完成后保存模型的状态字典
model_save_path = '/kaggle/working/weights.pth'
torch.save(model.state_dict(), model_save_path)

In [ ]:
test_df = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
batch_size = 100  # 可以根据你的内存大小调整
total_samples = test_df.shape[0]
predictions = []

for start in range(0, total_samples, batch_size):
    end = start + batch_size
    batch = test_df.iloc[start:end]
    batch_predictions = process_batch(batch, model, tokenizer, device)
    predictions.extend(batch_predictions)

# 将预测结果添加到 DataFrame
test_df['generated'] = predictions
submission_df = test_df[['id', 'generated']]
submission_df.to_csv('/kaggle/working/submission.csv', index=False)


In [ ]:
print(submission_df)